<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

# Assignment:

- Replicate the lesson code.

 - This means that if you haven't followed along already, type out the things that we did in class. Forcing your fingers to hit each key will help you internalize the syntax of what we're doing. Make sure you understand each line of code that you're writing, google things that you don't fully understand.
 - [Lambda Learning Method for DS - By Ryan Herr](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit?usp=sharing)
- Convert the `term` column from string to integer.
- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.
- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
##### Begin Working Here #####
import pandas as pd

In [0]:
# reading in the dataset 
source_url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/Ames%20Housing%20Data/train.csv'
iowa = pd.read_csv(source_url)

In [4]:
# random 5 rows to check give data a quick glance 
iowa.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
994,995,20,RL,96.0,12456,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,NridgHt,Norm,Norm,1Fam,1Story,10,5,2006,2007,Hip,CompShg,CemntBd,CmentBd,Stone,230.0,Ex,TA,PConc,Ex,TA,Gd,GLQ,1172,Unf,0,528,1700,GasA,...,Y,SBrkr,1718,0,0,1718,1,0,2,0,3,1,Ex,7,Typ,1,Gd,Attchd,2008.0,Fin,3,786,TA,TA,Y,216,48,0,0,0,0,NaN,NaN,NaN,0,7,2009,WD,Normal,337500
762,763,60,FV,72.0,8640,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,2Story,7,5,2009,2009,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,Mn,GLQ,24,Unf,0,732,756,GasA,...,Y,SBrkr,764,783,0,1547,0,0,2,1,3,1,Gd,7,Typ,0,NaN,Attchd,2009.0,Unf,2,614,TA,TA,Y,169,45,0,0,0,0,NaN,NaN,NaN,0,6,2010,Con,Normal,215200
149,150,50,RM,NaN,6240,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1.5Fin,5,4,1936,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,Gd,TA,No,Unf,0,Unf,0,896,896,GasA,...,Y,FuseA,896,448,0,1344,0,0,1,0,3,1,TA,7,Typ,0,NaN,Detchd,1936.0,Unf,1,240,Fa,TA,Y,200,114,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal,115000
1235,1236,70,RL,96.0,13132,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,5,5,1914,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,Gd,TA,Mn,Unf,0,Unf,0,747,747,GasA,...,Y,FuseF,892,892,0,1784,0,0,1,1,4,1,TA,9,Typ,0,NaN,Detchd,1914.0,Unf,1,180,Fa,Fa,N,203,40,0,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Normal,138887
799,800,50,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Feedr,Norm,1Fam,1.5Fin,5,7,1937,1950,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,252.0,TA,TA,BrkTil,Gd,TA,No,ALQ,569,Unf,0,162,731,GasA,...,Y,SBrkr,981,787,0,1768,1,0,1,1,3,1,Gd,7,Typ,2,TA,Detchd,1939.0,Unf,1,240,TA,TA,Y,0,0,264,0,0,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,175000


In [5]:
# Checking the rows and columns of the data
iowa.shape

(1460, 81)

In [6]:
# getting descriptive stats on two columns 
iowa[['YearBuilt', 'MSSubClass']].describe()

,YearBuilt,MSSubClass
count,1460.000000,1460.000000
mean,1971.267808,56.897260
std,30.202904,42.300571
min,1872.000000,20.000000
25%,1954.000000,20.000000
50%,1973.000000,50.000000
75%,2000.000000,70.000000
max,2010.000000,190.000000


In [7]:
# setting so that all rows show not truncated 
pd.set_option('display.max_rows', 100)
# Checking the data type of the first 4 columns 
iowa.dtypes[:4]

Id               int64
MSSubClass       int64
MSZoning        object
LotFrontage    float64
dtype: object

In [8]:
#Looking at the data type in BedroomAbvGr
#should be an integer from look in sample of data  
iowa['BedroomAbvGr'].dtype

dtype('int64')

In [9]:
# Calling column with the dot method (limited functionality)
#Users of this should be publicly shamed =) 
iowa.Fireplaces.sample(4)

1341    0
136     1
700     1
1355    1
Name: Fireplaces, dtype: int64

In [10]:
#calling with brackets method
#Same results but much more functionality 
iowa['Fireplaces'].sample(4)

697    0
622    0
24     1
448    1
Name: Fireplaces, dtype: int64

In [11]:
# example of functionality 
# dot method can't take cloumns with spaces in between name
# dot method can't select two different columns  
iowa[['Fire Places','BedroomAbvGr']].sample(4)

,Fireplaces,BedroomAbvGr
158,1,3
1325,0,2
1064,1,3
300,2,2


In [14]:
# checking to see if dtype matches what we see in LotFrontage 
# we see it was coded as a float when it should be an integer 
iowa['LotFrontage'].head()

0    65.0
1    80.0
2    68.0
3    60.0
4    84.0
Name: LotFrontage, dtype: float64

In [15]:
# checking to see the most common occurance in data 
# Will help determine why the data was casted to a float 
iowa['LotFrontage'].value_counts(dropna=False)

NaN      259
60.0     143
70.0      70
80.0      69
50.0      57
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 111, dtype: int64

In [16]:
# NaN is the most common dtype
# loading in nan from numpy so we can explore it 
import numpy as np
np.nan

nan

In [17]:
# Checking what dtype nan is for more insight into the problem 
# we find that nan is a float so all the data in the column is a float 
# dataframes try to keep all the same dtypes in a cloumn 
type(np.nan)

float

In [18]:
#checking to see how much of the data in LotFrontage is missing 
iowa['LotFrontage'].isnull().sum()

259

In [19]:
#compairing this to how many rows are in the whole dataframe 
#always good to ensure we're not deleting too much data from the set 
iowa.shape

(1460, 81)

In [0]:
# dropping the missing values from the dataframe 
iowa['LotFrontage'].dropna(inplace=True)

In [21]:
#Checking to ensure it was actually dropped 
iowa['LotFrontage'].isnull().sum()

0

# Stretch Goals

You can do more with the LendingClub or Instacart datasets.

LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.
- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 
- Take initiatve and work on your own ideas!

Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01